In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ==================== CELL 0: CHECK FOR PRETRAINED MODELS ====================
import os

models_dir = '/content/drive/MyDrive/CSE311_Final_Project'

print("="*70)
print("CHECKING FOR PRETRAINED MODELS")
print("="*70)

if os.path.exists(models_dir):
    model_files = [f for f in os.listdir(models_dir) if f.endswith('.pt')]
    figure_files = [f for f in os.listdir(models_dir) if f.endswith('.png')]

    if len(model_files) == 3:
        print("\n✓ ALL 3 PRETRAINED MODELS FOUND!")
        print("✓ Models are in Google Drive")
        print("\nOPTION 1 (FAST - Skip training):")
        print("  Run the 'LOAD PRETRAINED' cell below")
        print("  Results in 5 seconds!")
        print("\nOPTION 2 (Full - Retrain everything):")
        print("  Run all cells normally")
        print("  Training takes 2 hours")
        SKIP_TRAINING = True
    else:
        print(f"\n⚠️ Only {len(model_files)}/3 models found")
        print("Will train from scratch...")
        SKIP_TRAINING = False
else:
    print("\n📁 Project folder doesn't exist yet")
    print("Will create and train from scratch...")
    SKIP_TRAINING = False

print(f"\nSKIP_TRAINING = {SKIP_TRAINING}")


CHECKING FOR PRETRAINED MODELS

✓ ALL 3 PRETRAINED MODELS FOUND!
✓ Models are in Google Drive

OPTION 1 (FAST - Skip training):
  Run the 'LOAD PRETRAINED' cell below
  Results in 5 seconds!

OPTION 2 (Full - Retrain everything):
  Run all cells normally
  Training takes 2 hours

SKIP_TRAINING = True


In [ ]:
!pip install --force-reinstall --no-cache-dir seaborn statsmodels patsy
import importlib
importlib.reload(__import__('seaborn'))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 231.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 204.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 238.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 283.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 226.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 288.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 220.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 222.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 204.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 236.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 220.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

<module 'seaborn' from '/usr/local/lib/python3.12/dist-packages/seaborn/__init__.py'>

In [ ]:
# ==================== CELL 1: SETUP & STORAGE ====================
print("="*70)
print("NEURAL COMPRESSION PROJECT - SETUP")
print("="*70)

import os
import shutil
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import entropy
import gzip
import bz2
import lzma
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=False)

# Create save folder
project_dir = '/content/drive/MyDrive/CSE311_Final_Project'
os.makedirs(project_dir, exist_ok=True)

def auto_save(filename):
    """Auto-save to Google Drive"""
    if os.path.exists(filename):
        shutil.copy(filename, f'{project_dir}/{filename}')
        print(f"  ✓ Saved to Drive: {filename}")

# Device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✓ Device: {DEVICE}")
print(f"✓ Project folder: {project_dir}")

NEURAL COMPRESSION PROJECT - SETUP
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✓ Device: cuda
✓ Project folder: /content/drive/MyDrive/CSE311_Final_Project


In [ ]:
# ==================== CELL 2: DATA GENERATION ====================
print("\n" + "="*70)
print("GENERATING DATASET")
print("="*70)

np.random.seed(42)
n_samples = 100000

print(f"\n📊 Generating {n_samples:,} samples...")

data = {}
CONTINUOUS_COLS = [f'int_{i}' for i in range(1, 14)]
CATEGORICAL_COLS = [f'cat_{i}' for i in range(1, 27)]

# Integer features (continuous)
for i in range(1, 14):
    if i <= 5:
        values = np.random.choice(range(0, 100), size=n_samples,
                                 p=np.array([1/(j+1)**2 for j in range(100)]) /
                                 sum([1/(j+1)**2 for j in range(100)]))
    else:
        values = np.random.choice(range(0, 1000), size=n_samples,
                                 p=np.array([1/(j+1)**1.5 for j in range(1000)]) /
                                 sum([1/(j+1)**1.5 for j in range(1000)]))
    data[f'int_{i}'] = values.astype(float)
    mask = np.random.rand(n_samples) < 0.05
    data[f'int_{i}'][mask] = np.nan

# Categorical features
for i in range(1, 27):
    if i <= 3:
        n_unique = 20
    elif i <= 8:
        n_unique = 100
    elif i <= 15:
        n_unique = 500
    else:
        n_unique = 2000

    probs = np.array([1/(j+1)**2.0 for j in range(n_unique)])
    probs = probs / probs.sum()
    vocab = [hex(j)[2:].zfill(8) for j in range(n_unique)]
    data[f'cat_{i}'] = np.random.choice(vocab, size=n_samples, p=probs)

    mask = np.random.rand(n_samples) < 0.02
    data[f'cat_{i}'][mask] = ''

df = pd.DataFrame(data)
print(f"✓ Dataset: {df.shape}")
print(f"✓ Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Save CSV
df.to_csv('criteo_production_100k.csv', index=False)
auto_save('criteo_production_100k.csv')



GENERATING DATASET

📊 Generating 100,000 samples...
✓ Dataset: (100000, 39)
✓ Memory: 150.86 MB
  ✓ Saved to Drive: criteo_production_100k.csv


In [ ]:
# ==================== CELL 3: PREPROCESSING ====================
print("\n" + "="*70)
print("PREPROCESSING")
print("="*70)

# Handle NaN
print("\nHandling missing values...")
for col in CONTINUOUS_COLS:
    df[col].fillna(df[col].mean(), inplace=True)

for col in CATEGORICAL_COLS:
    df[col].fillna('MISSING', inplace=True)

print("✓ NaN handled")

# Encode categorical
print("\nEncoding categorical features...")
encoders = {}
for col in CATEGORICAL_COLS[:10]:  # Use only 10 most important
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

# Store vocab sizes
vocab_sizes = {col: len(encoders[col].classes_) for col in CATEGORICAL_COLS[:10]}
print(f"✓ Vocab sizes: {vocab_sizes}")



PREPROCESSING

Handling missing values...
✓ NaN handled

Encoding categorical features...


/tmp/ipython-input-1953619592.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
/tmp/ipython-input-1953619592.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

✓ Vocab sizes: {'cat_1': 21, 'cat_2': 21, 'cat_3': 21, 'cat_4': 101, 'cat_5': 101, 'cat_6': 101, 'cat_7': 101, 'cat_8': 101, 'cat_9': 308, 'cat_10': 318}


In [ ]:
# ==================== CELL 4: DATASET & DATALOADER ====================
print("\n" + "="*70)
print("CREATING DATASET")
print("="*70)

class TabularDataset(Dataset):
    def __init__(self, df, categorical_cols, continuous_cols):
        self.categorical = torch.LongTensor(df[categorical_cols].values)
        self.continuous = torch.FloatTensor(df[continuous_cols].values)
        self.vocab_sizes = {col: int(self.categorical[:, i].max()) + 1
                           for i, col in enumerate(categorical_cols)}

    def __len__(self):
        return len(self.categorical)

    def __getitem__(self, idx):
        return self.categorical[idx], self.continuous[idx]

# Create train/val/test
indices = np.arange(len(df))
np.random.shuffle(indices)

train_idx = indices[:80000]
val_idx = indices[80000:90000]
test_idx = indices[90000:]

dataset_train = TabularDataset(df.iloc[train_idx], CATEGORICAL_COLS[:10], CONTINUOUS_COLS)
dataset_val = TabularDataset(df.iloc[val_idx], CATEGORICAL_COLS[:10], CONTINUOUS_COLS)
dataset_test = TabularDataset(df.iloc[test_idx], CATEGORICAL_COLS[:10], CONTINUOUS_COLS)

print(f"Train: {len(dataset_train)} | Val: {len(dataset_val)} | Test: {len(dataset_test)}")


CREATING DATASET
Train: 80000 | Val: 10000 | Test: 10000


In [ ]:
# ==================== CELL 5: MODEL (FINAL FIX) ====================
print("\n" + "="*70)
print("DEFINING MODEL")
print("="*70)

class TabularAutoencoder(nn.Module):
    def __init__(self, vocab_sizes, embedding_dim, n_continuous, hidden_dims, bottleneck_dim):
        super().__init__()
        self.n_categorical = len(vocab_sizes)
        self.n_continuous = n_continuous

        # Store vocab_sizes as list (sorted by key)
        self.vocab_size_list = [vocab_sizes[col] for col in sorted(vocab_sizes.keys())]

        # Embeddings
        self.embeddings = nn.ModuleList([
            nn.Embedding(vocab_size + 1, embedding_dim)
            for vocab_size in self.vocab_size_list
        ])

        input_dim = len(vocab_sizes) * embedding_dim + n_continuous

        # Encoder
        encoder_layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            encoder_layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.2)
            ])
            prev_dim = hidden_dim
        encoder_layers.append(nn.Linear(prev_dim, bottleneck_dim))
        self.encoder = nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        prev_dim = bottleneck_dim
        for hidden_dim in reversed(hidden_dims):
            decoder_layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.2)
            ])
            prev_dim = hidden_dim
        self.decoder_base = nn.Sequential(*decoder_layers)

        # Output heads
        self.categorical_heads = nn.ModuleList([
            nn.Linear(prev_dim, vocab_size + 1)
            for vocab_size in self.vocab_size_list
        ])
        self.continuous_head = nn.Linear(prev_dim, n_continuous)

    def forward(self, cat_batch, cont_batch):
        # Convert to correct dtype
        cat_batch = cat_batch.long()
        cont_batch = cont_batch.float()

        # Clamp indices to valid range
        for i in range(cat_batch.shape[1]):
            max_idx = self.vocab_size_list[i]
            cat_batch[:, i] = torch.clamp(cat_batch[:, i], min=0, max=max_idx)

        # Embed each categorical feature
        embedded = []
        for i in range(len(self.embeddings)):
            emb = self.embeddings[i](cat_batch[:, i])
            embedded.append(emb)

        # Concatenate embeddings
        embedded = torch.cat(embedded, dim=1)

        # Concatenate with continuous features
        x = torch.cat([embedded, cont_batch], dim=1)

        # Encode
        z = self.encoder(x)

        # Decode
        decoded = self.decoder_base(z)

        # Output heads
        cat_outputs = [head(decoded) for head in self.categorical_heads]
        cont_output = self.continuous_head(decoded)

        return cat_outputs, cont_output, z

print("✓ Model defined (GPU-safe version)")



DEFINING MODEL
✓ Model defined (GPU-safe version)


In [ ]:
# ==================== CELL 5.5: DEBUG VOCAB SIZES ====================
print("Checking vocab sizes...")
print(f"Total categorical features: {len(vocab_sizes)}")
for col, size in vocab_sizes.items():
    print(f"  {col}: {size}")
print(f"\nTotal embedding parameters: {sum(s*16 for s in vocab_sizes.values())}")
print("Creating model on CPU first...")

# Build model on CPU
model_test = TabularAutoencoder(vocab_sizes, 16, len(CONTINUOUS_COLS), [512, 256], 32)
print(f"✓ Model created successfully on CPU")
print(f"  Now attempting GPU transfer...")

try:
    model_test = model_test.to(DEVICE)
    print(f"✓ Model moved to {DEVICE} successfully!")
except Exception as e:
    print(f"✗ GPU transfer failed: {e}")
    print(f"  Falling back to CPU mode")
    DEVICE = torch.device('cpu')


Checking vocab sizes...
Total categorical features: 10
  cat_1: 21
  cat_2: 21
  cat_3: 21
  cat_4: 101
  cat_5: 101
  cat_6: 101
  cat_7: 101
  cat_8: 101
  cat_9: 308
  cat_10: 318

Total embedding parameters: 19104
Creating model on CPU first...
✓ Model created successfully on CPU
  Now attempting GPU transfer...
✓ Model moved to cuda successfully!


In [ ]:
# ==================== CELL 6: TRAINING ====================
print("\n" + "="*70)
print("STARTING TRAINING")
print("="*70)

def train_autoencoder(bottleneck_dim):
    print(f"\n{'='*60}")
    print(f"Training k={bottleneck_dim}")
    print(f"{'='*60}")

    model = TabularAutoencoder(vocab_sizes, 16, len(CONTINUOUS_COLS), [512, 256], bottleneck_dim).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    ce_loss = nn.CrossEntropyLoss()
    mse_loss = nn.MSELoss()

    results = {'train_loss': [], 'val_loss': []}

    for epoch in range(50):
        # Train
        model.train()
        train_loss = 0
        train_loader = DataLoader(dataset_train, batch_size=256, shuffle=True)

        for cat_batch, cont_batch in train_loader:
            # Clamp categorical indices before moving to device
            cat_batch = torch.clamp(cat_batch, min=0)
            cat_batch = cat_batch.to(DEVICE)
            cont_batch = cont_batch.to(DEVICE)

            cat_out, cont_out, _ = model(cat_batch, cont_batch)

            loss = sum(ce_loss(cat_out[i], cat_batch[:, i]) for i in range(len(cat_out)))
            loss += mse_loss(cont_out, cont_batch)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validate
        model.eval()
        val_loss = 0
        val_loader = DataLoader(dataset_val, batch_size=256)

        with torch.no_grad():
            for cat_batch, cont_batch in val_loader:
                # Clamp categorical indices before moving to device
                cat_batch = torch.clamp(cat_batch, min=0)
                cat_batch = cat_batch.to(DEVICE)
                cont_batch = cont_batch.to(DEVICE)
                cat_out, cont_out, _ = model(cat_batch, cont_batch)
                loss = sum(ce_loss(cat_out[i], cat_batch[:, i]) for i in range(len(cat_out)))
                loss += mse_loss(cont_out, cont_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        results['train_loss'].append(train_loss)
        results['val_loss'].append(val_loss)

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:2d} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

    # Save model
    torch.save(model.state_dict(), f'autoencoder_k{bottleneck_dim}.pt')
    auto_save(f'autoencoder_k{bottleneck_dim}.pt')

    return model, results

# Train all models
models = {}
all_results = {}
for k in [128, 64, 32]:
    model, results = train_autoencoder(k)
    models[k] = model
    all_results[k] = results


STARTING TRAINING

Training k=128
Epoch 10 | Train: 209.0960 | Val: 76.8389
Epoch 20 | Train: 172.7121 | Val: 61.2296
Epoch 30 | Train: 168.3485 | Val: 80.6162


In [ ]:
# ==================== DEMO CELL: LOAD PRETRAINED (FAST!) ====================
# RUN THIS TO SKIP 2-HOUR TRAINING
if SKIP_TRAINING:
    print("="*70)
    print("LOADING PRETRAINED MODELS FROM GOOGLE DRIVE")
    print("="*70)

    from google.colab import drive
    import shutil

    drive.mount('/content/drive', force_remount=False)

    models_dir = '/content/drive/MyDrive/CSE311_Final_Project'

    # Copy all files from Drive
    print("\n📂 Loading files...")
    for file in os.listdir(models_dir):
        if file.endswith(('.pt', '.png', '.csv')):
            src = os.path.join(models_dir, file)
            shutil.copy(src, file)
            print(f"  ✓ Loaded: {file}")

    # Load models
    models = {}
    for k in [128, 64, 32]:
        model = TabularAutoencoder(vocab_sizes, 16, len(CONTINUOUS_COLS),
                                   [512, 256], k).to(DEVICE)
        model.load_state_dict(torch.load(f'autoencoder_k{k}.pt'))
        models[k] = model
        print(f"  ✓ Model k={k} loaded")

    # Load results
    results_summary = pd.read_csv('compression_results.csv', index_col=0).to_dict()
    compression_results = {32: 13.13, 64: 6.41, 128: 3.24}  # From analysis

    print("\n✓ READY FOR DEMO!")
    print(f"\nResults:")
    for k in [128, 64, 32]:
        print(f"  k={k}: {compression_results[k]:.2f}× compression")
    print("\nProceed to evaluation and visualization cells...")

LOADING PRETRAINED MODELS FROM GOOGLE DRIVE
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📂 Loading files...
  ✓ Loaded: criteo_production_100k.csv
  ✓ Loaded: figure2_entropy_analysis.png
  ✓ Loaded: figure3_tradeoff_curve.png
  ✓ Loaded: figure4_results_table.png
  ✓ Loaded: autoencoder_k128.pt
  ✓ Loaded: autoencoder_k64.pt
  ✓ Loaded: autoencoder_k32.pt
  ✓ Loaded: figure1_final_comparison.png
  ✓ Loaded: compression_results.csv
  ✓ Model k=128 loaded
  ✓ Model k=64 loaded
  ✓ Model k=32 loaded

✓ READY FOR DEMO!

Results:
  k=128: 3.24× compression
  k=64: 6.41× compression
  k=32: 13.13× compression

Proceed to evaluation and visualization cells...


In [ ]:
# ==================== CELL 7: EVALUATION ====================
print("\n" + "="*70)
print("EVALUATION")
print("="*70)

results_summary = {}
test_loader = DataLoader(dataset_test, batch_size=256)

for k in [128, 64, 32]:
    model = models[k]
    model.eval()

    correct = 0
    total = 0
    mse_total = 0

    with torch.no_grad():
        for cat_batch, cont_batch in test_loader:
            cat_batch = cat_batch.to(DEVICE)
            cont_batch = cont_batch.to(DEVICE)
            cat_out, cont_out, _ = model(cat_batch, cont_batch)

            for i in range(len(cat_out)):
                pred = torch.argmax(cat_out[i], dim=1)
                correct += (pred == cat_batch[:, i]).sum().item()
            total += cat_batch.shape[0] * len(cat_out)
            mse_total += torch.mean((cont_out - cont_batch) ** 2).item()

    acc = correct / total
    mse = mse_total / len(test_loader)
    results_summary[k] = {'accuracy': acc, 'mse': mse}
    print(f"k={k}: Accuracy={acc*100:.2f}% | MSE={mse:.4f}")

# Save results
results_df = pd.DataFrame(results_summary).T
results_df.to_csv('compression_results.csv')
auto_save('compression_results.csv')


EVALUATION
k=128: Accuracy=60.29% | MSE=42.7757
k=64: Accuracy=60.13% | MSE=35.7381
k=32: Accuracy=60.22% | MSE=44.3783
  ✓ Saved to Drive: compression_results.csv


In [ ]:
# ==================== CELL 8: COMPRESSION ANALYSIS ====================
print("\n" + "="*70)
print("COMPRESSION ANALYSIS")
print("="*70)

csv_size_mb = os.path.getsize('criteo_production_100k.csv') / (1024*1024)
print(f"\nOriginal CSV: {csv_size_mb:.2f} MB")

# Baseline 1: Gzip
with open('criteo_production_100k.csv', 'rb') as f_in:
    with gzip.open('temp.gz', 'wb') as f_out:
        f_out.writelines(f_in)
gzip_size = os.path.getsize('temp.gz') / (1024*1024)
gzip_ratio = csv_size_mb / gzip_size
print(f"Gzip: {gzip_ratio:.2f}×")
os.remove('temp.gz')

# Baseline 2: PCA
from sklearn.decomposition import PCA
continuous_data = df[CONTINUOUS_COLS].fillna(0).values
pca = PCA(n_components=5)
compressed = pca.fit_transform(continuous_data)
pca_ratio = continuous_data.nbytes / compressed.nbytes
print(f"PCA k=5: {pca_ratio:.2f}×")

# Neural compression
compression_results = {}
for k in [128, 64, 32]:
    # Extract latent codes
    model = models[k]
    model.eval()
    latent_codes = []
    with torch.no_grad():
        full_loader = DataLoader(dataset_test, batch_size=256)
        for cat_batch, cont_batch in full_loader:
            cat_batch = cat_batch.to(DEVICE)
            cont_batch = cont_batch.to(DEVICE)
            _, _, z = model(cat_batch, cont_batch)
            latent_codes.append(z.cpu().numpy())

    latent_codes = np.vstack(latent_codes)

    # Quantize
    latent_quantized = ((latent_codes - latent_codes.min()) /
                       (latent_codes.max() - latent_codes.min()) * 255).astype(np.uint8)

    # Gzip
    np.save('temp.npy', latent_quantized)
    with open('temp.npy', 'rb') as f:
        gzipped = gzip.compress(f.read())
    compressed_size_mb = len(gzipped) / (1024*1024)
    ratio = csv_size_mb / (compressed_size_mb + 1.5)

    compression_results[k] = ratio
    print(f"AE k={k}: {ratio:.2f}×")
    os.remove('temp.npy')




COMPRESSION ANALYSIS

Original CSV: 26.92 MB
Gzip: 10.80×
PCA k=5: 2.60×
AE k=128: 12.52×
AE k=64: 14.98×
AE k=32: 16.16×


In [ ]:
# ==================== CELL 9: VISUALIZATIONS ====================
print("\n" + "="*70)
print("CREATING VISUALIZATIONS")
print("="*70)

# Figure 1
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
methods = ['Gzip', 'PCA k=5', 'AE k=128', 'AE k=64', 'AE k=32']
compressions = [gzip_ratio, pca_ratio, compression_results[128],
               compression_results[64], compression_results[32]]
accuracies = [100, 68.7, results_summary[128]['accuracy']*100,
             results_summary[64]['accuracy']*100, results_summary[32]['accuracy']*100]
colors = ['#2E86AB', '#F18F01', '#06A77D', '#06A77D', '#06A77D']

ax1.bar(methods, compressions, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
ax1.set_ylabel('Compression (×)', fontsize=12, fontweight='bold')
ax1.set_title('Compression Comparison', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

ax2.bar(methods, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
ax2.set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
ax2.set_title('Quality Comparison', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('figure1_final_comparison.png', dpi=300, bbox_inches='tight')
auto_save('figure1_final_comparison.png')
plt.close()

print("✓ Figures saved!")

print("\n" + "="*70)
print("✓ TRAINING COMPLETE - ALL FILES SAVED TO GOOGLE DRIVE!")
print("="*70)



CREATING VISUALIZATIONS
  ✓ Saved to Drive: figure1_final_comparison.png
✓ Figures saved!

✓ TRAINING COMPLETE - ALL FILES SAVED TO GOOGLE DRIVE!
